In [4]:
import pandas as pd
import numpy as np

In [225]:
df = pd.read_csv('C:/Users/User/Documents/GitHub/AnotherTown/Data/Cleaned/Safety_Raw.csv')
df = df.drop(df[df['County'].isna()].index, axis=0)
df.head()

,FIPS,State,County,Firearm Fatalities Rate,Suicide Rate (Age-Adjusted),Homicide Rate,Juvenile Arrest Rate,Drug Overdose Mortality Rate,Injury Death Rate,Violent Crime Rate,% Driving Deaths with Alcohol Involvement,% Excessive Drinking
1,1001,Alabama,Autauga,16.0,18.0,5.0,11.0,11.0,69.0,272.0,27.0,15
2,1003,Alabama,Baldwin,14.0,19.0,3.0,26.0,14.0,72.0,204.0,31.0,18
3,1005,Alabama,Barbour,18.0,13.0,8.0,15.0,NaN,70.0,414.0,40.0,13
4,1007,Alabama,Bibb,24.0,21.0,8.0,NaN,28.0,108.0,89.0,28.0,16
5,1009,Alabama,Blount,20.0,17.0,6.0,7.0,24.0,107.0,483.0,19.0,14


In [226]:
df['Firearm Fatalities Rate'] = df['Firearm Fatalities Rate'].astype('float64')

for c, col in enumerate(df.columns[3:]):
    df[col] = pd.qcut(df[col].rank(method='first'), 10, labels=[i for i in range(1,11)]) 
df.head()

,FIPS,State,County,Firearm Fatalities Rate,Suicide Rate (Age-Adjusted),Homicide Rate,Juvenile Arrest Rate,Drug Overdose Mortality Rate,Injury Death Rate,Violent Crime Rate,% Driving Deaths with Alcohol Involvement,% Excessive Drinking
1,1001,Alabama,Autauga,6,6,5,2,2,3,7,5,2
2,1003,Alabama,Baldwin,5,7,2,6,3,3,5,7,5
3,1005,Alabama,Barbour,7,2,8,3,NaN,3,9,9,1
4,1007,Alabama,Bibb,9,8,8,NaN,8,9,2,5,3
5,1009,Alabama,Blount,8,5,6,1,7,9,9,3,2


In [227]:
weights_raw = [5,1,10,8,3,3,10,2,1]
weights = [i/sum(weights_raw) for i in weights_raw]

nans = pd.isna(df).any(axis=1)

def zero_weight(zero_idx):
    w = weights
    w = [0  if idx + 3 in zero_idx else weight for idx, weight in enumerate(weights)]
    return np.divide(w, sum(w))
    
#re-weight if nan in row
temp = []
for i, row in df.iterrows():
    w = weights
    if nans[i]:
        w = zero_weight([c for c, i in enumerate(row) if c > 2 and not i >= 0])
    temp.append(sum(list((row[3:] * w).fillna(0))))
                
df['Safety_avg'] = temp

In [228]:
df = df.dropna(thresh=10, axis=0)
df.head()

,FIPS,State,County,Firearm Fatalities Rate,Suicide Rate (Age-Adjusted),Homicide Rate,Juvenile Arrest Rate,Drug Overdose Mortality Rate,Injury Death Rate,Violent Crime Rate,% Driving Deaths with Alcohol Involvement,% Excessive Drinking,Safety_avg
1,1001,Alabama,Autauga,6,6,5,2,2,3,7,5,2,4.627907
2,1003,Alabama,Baldwin,5,7,2,6,3,3,5,7,5,4.348837
3,1005,Alabama,Barbour,7,2,8,3,NaN,3,9,9,1,6.475000
4,1007,Alabama,Bibb,9,8,8,NaN,8,9,2,5,3,6.200000
5,1009,Alabama,Blount,8,5,6,1,7,9,9,3,2,6.023256


In [229]:
df.sort_values(by=['Safety_avg'])

,FIPS,State,County,Firearm Fatalities Rate,Suicide Rate (Age-Adjusted),Homicide Rate,Juvenile Arrest Rate,Drug Overdose Mortality Rate,Injury Death Rate,Violent Crime Rate,% Driving Deaths with Alcohol Involvement,% Excessive Drinking,Safety_avg
596,16065,Idaho,Madison,1,1,NaN,NaN,NaN,1,1,1,3,1.090909
740,18057,Indiana,Hamilton,1,1,1,2,2,1,1,3,8,1.511628
455,13117,Georgia,Forsyth,2,3,1,NaN,2,1,1,3,6,1.542857
2296,42027,Pennsylvania,Centre,1,1,1,1,1,1,2,5,10,1.627907
1818,34027,New Jersey,Morris,1,1,1,2,5,1,1,2,7,1.651163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2050,Alaska,Bethel,10,10,9,10,NaN,10,NaN,10,10,9.666667
83,2158,Alaska,Kusilvak,10,10,10,NaN,NaN,10,NaN,10,4,9.727273
1626,30003,Montana,Big Horn,10,10,10,9,NaN,10,10,10,8,9.750000
86,2180,Alaska,Nome,10,10,NaN,10,NaN,10,NaN,10,10,10.000000


In [230]:
df[df['County'].isin(['San Diego', 'Los Angeles', 'Ventura'])]

,FIPS,State,County,Firearm Fatalities Rate,Suicide Rate (Age-Adjusted),Homicide Rate,Juvenile Arrest Rate,Drug Overdose Mortality Rate,Injury Death Rate,Violent Crime Rate,% Driving Deaths with Alcohol Involvement,% Excessive Drinking,Safety_avg
209,6037,California,Los Angeles,1,1,6,1,1,1,9,5,7,4.348837
227,6073,California,San Diego,1,2,2,1,3,1,8,6,10,3.465116
246,6111,California,Ventura,1,1,2,3,3,1,6,8,7,3.372093


In [231]:
df2 = df[['FIPS', 'State', 'County', 'Safety_avg']]
df2.head()

,FIPS,State,County,Safety_avg
1,1001,Alabama,Autauga,4.627907
2,1003,Alabama,Baldwin,4.348837
3,1005,Alabama,Barbour,6.475000
4,1007,Alabama,Bibb,6.200000
5,1009,Alabama,Blount,6.023256


In [232]:
df2.to_csv('C:/Users/User/Documents/GitHub/AnotherTown/Data/Cleaned/Safety_Scores.csv')